In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from itertools import combinations
from collections import Counter

In [2]:
# Загружаем данные
data = pd.read_csv('movie_bd_v5.csv')

data.sample(5)  # посмотрим на данные

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
23,tt2322441,40000000,569651467,Fifty Shades of Grey,Dakota Johnson|Jamie Dornan|Jennifer Ehle|Eloi...,Sam Taylor-Johnson,Are you curious?,When college senior Anastasia Steele steps in ...,125,Drama|Romance,Focus Features|Trigger Street Productions|Mich...,2/11/2015,5.3,2015
397,tt0814255,95000000,226497209,Percy Jackson & the Olympians: The Lightning T...,Logan Lerman|Brandon T. Jackson|Alexandra Dadd...,Chris Columbus,Worlds Collide,"Accident prone teenager, Percy discovers he's ...",118,Adventure|Fantasy|Family,Ingenious Film Partners|1492 Pictures|Dune Ent...,2/1/2010,6.0,2010
1458,tt0414982,25000000,117719158,Final Destination 3,Mary Elizabeth Winstead|Ryan Merriman|Kris Lem...,James Wong,This Ride Will Be The Death Of You.,A student's premonition of a deadly rollercoas...,93,Horror|Mystery,New Line Cinema|Zide-Perry Productions|Kumar M...,2/10/2006,5.8,2006
1666,tt0359517,12000000,31179516,Johnson Family Vacation,Cedric the Entertainer|Vanessa Williams|Steve ...,Christopher Erskin,Take the ride.,AAA can't help the roadside emergency that is ...,97,Comedy|Family,Fox Searchlight Pictures|Bird and a Bear Enter...,4/7/2004,5.3,2004
1094,tt0328828,55000000,231449203,American Wedding,Jason Biggs|Alyson Hannigan|Seann William Scot...,Jesse Dylan,Forever hold your piece.,"With high school a distant memory, Jim and Mic...",103,Comedy|Romance,Universal Pictures|Zide-Perry Productions|Live...,8/1/2003,6.0,2003


In [6]:
data.describe() # просмотр базовых статистик в числовых колонках

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [3]:
answers = {}  # создадим словарь для ответов


# расчитываем прибыль/убытки
data['profit'] = data.revenue - data.budget

# Переводим дату релиза в формат даты
data['release_date'] = pd.to_datetime(data.release_date)

# Расчет длины названия
data['title_len'] = data.original_title.map(lambda x: len(x))

# Расчет длины описания
data['overview_len'] = data.overview.map(lambda x: len(x))

# Получаем лист с именами актеров в отдельной колонке (для вопроса 27)
data['cast_split'] = data.cast.str.split('|')

# Создаем комбинации из имен актеров (для вопроса 27)
data['cast_together'] = data.cast_split.apply(lambda x: list(combinations(x, 2)))


data.sample(5)


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,title_len,overview_len,cast_split,cast_together
1186,tt1170358,250000000,958400000,The Hobbit: The Desolation of Smaug,Martin Freeman|Ian McKellen|Richard Armitage|K...,Peter Jackson,Beyond darkness... beyond desolation... lies t...,"The Dwarves, Bilbo and Gandalf have successful...",161,Adventure|Fantasy,WingNut Films|New Line Cinema|Metro-Goldwyn-Ma...,2013-12-11,7.6,2013,708400000,35,191,"[Martin Freeman, Ian McKellen, Richard Armitag...","[(Martin Freeman, Ian McKellen), (Martin Freem..."
1110,tt0311113,150000000,212011111,Master and Commander: The Far Side of the World,Russell Crowe|Paul Bettany|James D'Arcy|Billy ...,Peter Weir,The Courage to Do the Impossible Lies in the H...,After an abrupt and violent encounter with a F...,138,Adventure,Universal Pictures|Twentieth Century Fox Film ...,2003-11-14,6.6,2003,62011111,47,370,"[Russell Crowe, Paul Bettany, James D'Arcy, Bi...","[(Russell Crowe, Paul Bettany), (Russell Crowe..."
1761,tt0419843,10000000,17500000,In the Land of Women,Meg Ryan|Kristen Stewart|Adam Brody|JoBeth Wil...,Jon Kasdan,Get ready to fall,After a bad breakup with his girlfriend leaves...,97,Romance|Comedy|Drama,Castle Rock Entertainment|Land Films Inc.|Anon...,2007-04-16,6.0,2007,7500000,20,231,"[Meg Ryan, Kristen Stewart, Adam Brody, JoBeth...","[(Meg Ryan, Kristen Stewart), (Meg Ryan, Adam ..."
1132,tt0349205,40000000,190212113,Cheaper by the Dozen,Steve Martin|Bonnie Hunt|Piper Perabo|Tom Well...,Shawn Levy,Growing pains? They've got twelve of them!,The Baker brood moves to Chicago after patriar...,98,Comedy|Drama|Family,Twentieth Century Fox Film Corporation|Robert ...,2003-12-24,5.8,2003,150212113,20,331,"[Steve Martin, Bonnie Hunt, Piper Perabo, Tom ...","[(Steve Martin, Bonnie Hunt), (Steve Martin, P..."
1480,tt0418689,90000000,65900249,Flags of Our Fathers,Ryan Phillippe|Adam Beach|Jesse Bradford|John ...,Clint Eastwood,They fight for their country but they die for ...,There were five Marines and one Navy Corpsman ...,132,War|Drama|History,DreamWorks SKG|Amblin Entertainment|Malpaso Pr...,2006-10-19,6.7,2006,-24099751,20,317,"[Ryan Phillippe, Adam Beach, Jesse Bradford, J...","[(Ryan Phillippe, Adam Beach), (Ryan Phillippe..."


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [4]:
data.original_title[data.budget == data.budget.max()].to_string(index=False)

' Pirates of the Caribbean: On Stranger Tides'

In [5]:
# в словарь вставляем номер вопроса и ваш ответ на него

# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'  # "+"


ВАРИАНТ 2

In [38]:
# можно добавлять разные варианты решения
data.query('budget == budget.max()').original_title


723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
data.query('runtime == runtime.max()').original_title


1157    Gods and Generals
Name: original_title, dtype: object

In [7]:
answers['2'] = 'Gods and Generals'  # "+"


# 3. Какой из фильмов самый короткий (в минутах)?





In [8]:
data['original_title'][data.runtime == min(data.runtime)]


768    Winnie the Pooh
Name: original_title, dtype: object

In [9]:
answers['3'] = 'Winnie the Pooh'  # "+"


# 4. Какова средняя длительность фильмов?


In [12]:
round(data.runtime.mean())


110

In [13]:
answers['4'] = '110'  # "+"


# 5. Каково медианное значение длительности фильмов? 

In [14]:
data.runtime.median()

107.0

In [15]:
answers['5'] = '107'  # "+"

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [16]:
data.original_title[data.profit == data.profit.max()]  # колонка profit расчитана в начале файла


239    Avatar
Name: original_title, dtype: object

In [17]:
answers['6'] = 'Avatar'  # "+"


# 7. Какой фильм самый убыточный? 

In [18]:
data.original_title[data.profit == data.profit.min()]


1245    The Lone Ranger
Name: original_title, dtype: object

In [20]:
answers['7'] = 'The Lone Ranger'  # "+"


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [21]:
len(data.query('revenue > budget'))


1478

In [22]:
answers['8'] = '1478'  # "+"


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [23]:
data.query('release_year == 2008').query('revenue == revenue.max()').original_title


599    The Dark Knight
Name: original_title, dtype: object

In [24]:
answers['9'] = 'The Dark Knight'  # "+"


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [25]:
data.query('release_year >= 2012 & release_year <= 2014').query('profit == profit.min()').original_title


1245    The Lone Ranger
Name: original_title, dtype: object

In [26]:
answers['10'] = 'The Lone Ranger'  # "+" 


# 11. Какого жанра фильмов больше всего?

In [27]:
genres = []  # создадим лист для записи жанров в каждой строке

# Парсим каждую запись в колонке genres
for genres_row in range(0, len(data)):
    genres_i = data.genres[genres_row].split(sep='|')
    for genre in range(0, len(genres_i)):
        genres.append(genres_i[genre])

# Расчитываем количество жанров в листе и выбираем жанр, встречающийся чаще всего
max(set(genres), key=genres.count)


'Drama'

In [28]:
answers['11'] = 'Drama'


ВАРИАНТ 2

In [29]:
# Вариант 2
max(set(data.genres.str.split('|').sum()), key=data.genres.str.split('|').sum().count)

# Вариант 3
Counter(data.genres.str.split('|').sum()).most_common(1)


[('Drama', 782)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [30]:
Counter(data.query('profit > 0').genres.str.split('|').sum()).most_common(1)


[('Drama', 560)]

In [31]:
answers['12'] = 'Drama'  # "+" 


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [32]:
data.groupby('director').revenue.sum().sort_values(ascending = False).reset_index().director[0]


'Peter Jackson'

In [34]:
answers['13'] = 'Peter Jackson'  # "+" 


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [40]:
Counter(data[data.genres.str.contains('Action')].director.str.split('|').sum()).most_common(1)


[('Robert Rodriguez', 9)]

In [41]:
answers['14'] = 'Robert Rodriguez'  # "+" 


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [42]:
data.query('release_year == 2012').query('revenue == revenue.max()').cast.str.split('|').sum()

['Robert Downey Jr.',
 'Chris Evans',
 'Mark Ruffalo',
 'Chris Hemsworth',
 'Scarlett Johansson']

In [44]:
answers['15'] = 'Chris Hemsworth'  # "+" 


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [45]:
Counter(data.query('budget > budget.mean()').cast.str.split('|').sum()).most_common(1)

[('Matt Damon', 18)]

In [46]:
answers['16'] = 'Matt Damon'  # "+" 

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [47]:
Counter(data[data.cast.str.contains('Nicolas Cage')].genres.str.split('|').sum()).most_common(1)

[('Action', 17)]

In [49]:
answers['17'] = 'Action'  # "+" 

# 18. Самый убыточный фильм от Paramount Pictures

In [50]:
data[data.production_companies.str.contains('Paramount Pictures')].query('profit == profit.min()').original_title


925    K-19: The Widowmaker
Name: original_title, dtype: object

In [51]:
answers['18'] = 'K-19: The Widowmaker'  # "+" 


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [52]:
data.groupby(['release_year']).revenue.sum().reset_index().max()


release_year           2015
revenue         25449202382
dtype: int64

In [53]:
answers['19'] = '2015'  # "+" 


# 20. Какой самый прибыльный год для студии Warner Bros?

In [54]:
# отдельный датасет с записями о Warner Bros
warner_profit = data[data.production_companies.str.contains('Warner Bros')]

warner_profit.groupby(['release_year']).profit.sum().reset_index().query('profit == profit.max()')


,release_year,profit
14,2014,2295464519


In [55]:
answers['20'] = '2014'  # "+" 


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [56]:
# Отдельный датафрейм со вычислением количества фильмов по годам
movies_cnt_by_year = data.groupby(data['release_date'].dt.month).size().reset_index()

# Переименуем колонки для удобства
movies_cnt_by_year.columns = ['release_date', 'cnt']

# Отфильтруем год с максимальным количеством фильмов
movies_cnt_by_year.query('cnt == cnt.max()')


,release_date,cnt
8,9,227


In [57]:
answers['21'] = 'Сентябрь'  # "+" 

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [58]:
len(data.query('release_date.dt.month in [6, 7, 8]'))


450

In [59]:
answers['22'] = '450'  # "+" 


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [60]:
Counter(data.query('release_date.dt.month in [1, 2, 12]').director.str.split('|').sum()).most_common(1)


[('Peter Jackson', 7)]

In [61]:
answers['23'] = 'Peter Jackson'  # "+" 


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [62]:
# уникальный список студий
companies = data.production_companies.str.split('|').explode().unique()

# серия, в которой будем записывать студию и длину названий
title_length = pd.Series(index = companies, dtype = 'float64') 

# вычислим среднюю длину названий для каждой студии
for company in companies:
    title_length[company] = data['title_len'][data['production_companies'].map(lambda x: True if company in x else False)].mean()

# ответ
title_length.sort_values(ascending = False).head(1)

Four By Two Productions    83.0
dtype: float64

In [63]:
answers['24'] = 'Four By Two Productions'  # "+" 


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [65]:
# серия, в которой будем записывать студию и длину описаний
overview_length = pd.Series(index = companies, dtype = 'float64') # серия, в которой будем записывать компанию и длину названий

# вычислим среднюю длину описаний для каждой студии
for company in companies:
    overview_length[company] = data['overview_len'][data['production_companies'].map(lambda x: True if company in x else False)].mean()
    
# ответ
overview_length.sort_values(ascending = False).head(1)


Midnight Picture Show    1000.0
dtype: float64

In [66]:
answers['25'] = 'Midnight Picture Show'  # "+" 


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [67]:
# Находим через квантиль
data.query('vote_average >= vote_average.quantile(0.99)').original_title.sort_values()


1191                                     12 Years a Slave
316                                              3 Idiots
124                                            Big Hero 6
1213                                   Dallas Buyers Club
1563                Eternal Sunshine of the Spotless Mind
128                                             Gone Girl
119                               Guardians of the Galaxy
1188                                                  Her
370                                             Inception
9                                              Inside Out
118                                          Interstellar
1800                                              Memento
283                                            Mr. Nobody
1205                                            Prisoners
34                                                   Room
28                                              Spotlight
599                                       The Dark Knight
177           

In [68]:
answers['26'] = ['Inside Out', 'The Dark Knight', ' 12 Years a Slave']  # "+" 


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [4]:
# Колонки созданы в начале файла
list1 = data.cast_together.tolist()  # склеиваем результат колонки cast_together в list of lists
list2 = list(itertools.chain(*list1))  # превращаем list of lists в единый лист

Counter(list2).most_common(5)  # расчитываем количество пар и выбираем наиболее часто встречающиеся

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7),
 (('Ben Stiller', 'Owen Wilson'), 6),
 (('Johnny Depp', 'Helena Bonham Carter'), 6)]

In [8]:
data.cast_together

0       [(Chris Pratt, Bryce Dallas Howard), (Chris Pr...
1       [(Tom Hardy, Charlize Theron), (Tom Hardy, Hug...
2       [(Shailene Woodley, Theo James), (Shailene Woo...
3       [(Harrison Ford, Mark Hamill), (Harrison Ford,...
4       [(Vin Diesel, Paul Walker), (Vin Diesel, Jason...
                              ...                        
1884    [(Patrick Stewart, Hugh Jackman), (Patrick Ste...
1885    [(Richard E. Grant, Jonathan Lipnicki), (Richa...
1886    [(Rene Russo, Jason Alexander), (Rene Russo, P...
1887    [(Meg Ryan, Diane Keaton), (Meg Ryan, Lisa Kud...
1888    [(Susan Ward, Lori Heuring), (Susan Ward, Matt...
Name: cast_together, Length: 1889, dtype: object

In [72]:
answers['27'] = ['Daniel Radcliffe', 'Rupert Grint']  # "+" 


ВАРИАНТ 2

# Submission

In [73]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': '110',
 '5': '107',
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': '1478',
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': '450',
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out', 'The Dark Knight', ' 12 Years a Slave'],
 '27': ['Daniel Radcliffe', 'Rupert Grint']}

In [74]:
# и убедиться что ни чего не пропустил)
len(answers)

27